In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [11]:
def likelihood(train_data,train_label,current_x):
    P0 = np.ones(train_data.shape[1])
    P1 = np.ones(train_data.shape[1])
    Count = Counter(train_label)
    Count_0 = Count[0]
    Count_1 = Count[1]

    for i in range(train_data.shape[1]):
        Con_0,Con_1 = 0,0
        for j in range(train_data.shape[0]):
            if train_label[j]==0 and current_x[i]==train_data[j,i]:
                Con_0 +=1
            elif train_label[j]==1 and current_x[i]==train_data[j,i]:
                Con_1 +=1
        
        P0[i] = Con_0/Count_0
        P1[i] = Con_1/Count_1
    
        
    
    
    return np.product(P0),np.product(P1)

In [12]:
def marginal_likelihood(train_data, current_x):
    Px = 1
    for i in range(train_data.shape[1]):
        count = Counter(train_data[:,i])
        Pxi = count[current_x[i]]/train_data.shape[0]
        Px *= Pxi  
    
    return Px

In [13]:
def prior_probability(train_label):
    
    total_class = Counter(train_label)
    
    P_0 = total_class[0]/len(train_label)
    P_1 = total_class[1]/len(train_label)

    
    return P_0,P_1

In [14]:
def posterior_probability(train_input,train_label,test_input):
    
    prior0,prior1= prior_probability(y_train)
    marginal = marginal_likelihood(train_input,test_input)
    likelihood_0,likelihood_1 = likelihood(train_input,y_train,test_input)
    posterior_0 =( prior0 * likelihood_0)/marginal
    posterior_1 =( prior1 * likelihood_1)/marginal
    P0 = posterior_0 / (posterior_0+posterior_1)
    P1 = posterior_1 / (posterior_0+posterior_1)

    return (P0,P1)

In [15]:
def naive_bayes(train_data,train_label,test_data):
    Probabilities = []
    y_pred = []

    for rows in test_data:
        Probabilities.append(posterior_probability(train_data,train_label,rows))
    for value0,value1 in Probabilities:
        if value0>value1:
            y_pred.append(0)
        else:
            y_pred.append(1)
    return Probabilities,y_pred
    

In [16]:
data = pd.read_csv('project3_dataset4.txt', sep='\t', header=None)
# lc = LabelEncoder()
# data.iloc[:,4]= lc.fit_transform(data.iloc[:,4])
data[0] = data[0].map( {'sunny':0, 'overcast':1, 'rain':2})
data[1] = data[1].map({'hot':0,'mild':1,'cool':2})
data[2] = data[2].map({'high':0,'normal':1})
data[3]= data[3].map({'weak':0,'strong':1})
rows,col=data.shape
x = data.iloc[:,:col-1].values
y = data.iloc[:,-1].values

In [17]:
# # for only one input
# x_train, x_test , y_train, y_test=  train_test_split(x,y,test_size = 0.2, random_state = 42)
# Probabilities, y_pred = naive_bayes(x_train, y_train, x_test )
# print(Probabilities)
# print(y_pred)
# print(y_test)
# p,r,f,s = score(y_test,y_pred)
# accuracy = accuracy_score(y_test,y_pred)
# print('precision:',p.mean())
# print('recall:',r.mean())
# print('f1measure',f.mean())
# print('accuracy',accuracy)

In [18]:
# for k fold
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
precision = np.zeros(10)
recall =  np.zeros(10)
fscore =  np.zeros(10)
accuracy =  np.zeros(10)
support = np.zeros(10)


kf = KFold(n_splits=10)
precision = np.zeros(10)
recall =  np.zeros(10)
fscore =  np.zeros(10)
accuracy =  np.zeros(10)
i = 0
for train_index, test_index in kf.split(x):
    X_train,X_test = x[train_index],x[test_index]
    y_train , y_test = y[train_index], y[test_index]
    Probability , y_pred = naive_bayes(X_train, y_train, X_test)
    print('Probability:',Probability)
    print('test_output:',y_pred)
    p,r,f,support = score(y_test,y_pred)
    accuracy[i] = accuracy_score(y_test,y_pred)
    precision[i] , recall[i], fscore[i] = p.mean(),r.mean(),f.mean()
    i +=1
    
print('precision:',precision.mean())
print('recall:',recall.mean())
print('f1measure',fscore.mean())
print('accuracy',accuracy.mean())


Probability: [(0.0, 1.0), (0.0, 1.0)]
test_output: [1, 1]
Probability: [(0.0, 1.0), (0.785346307956497, 0.21465369204350304)]
test_output: [1, 0]
Probability: [(0.0, 1.0), (0.0, 1.0)]
test_output: [1, 1]
Probability: [(0.0, 1.0), (0.25, 0.75)]
test_output: [1, 1]
Probability: [(0.32954301652006013, 0.67045698347994)]
test_output: [1]
Probability: [(0.17928740260877185, 0.8207125973912281)]
test_output: [1]
Probability: [(0.7107820453493753, 0.2892179546506247)]
test_output: [0]
Probability: [(0.0, 1.0)]
test_output: [1]
Probability: [(0.0, 1.0)]
test_output: [1]
Probability: [(0.3875598086124402, 0.6124401913875597)]
test_output: [1]
precision: 0.5
recall: 0.525
f1measure 0.5
accuracy 0.55


C:\Users\sachi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\sachi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
